# Functions

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import random
from sklearn.preprocessing import *

pd.options.mode.chained_assignment = None 
import warnings
from sklearn.linear_model import LassoCV
from sklearn.linear_model import lasso_path
from tqdm import tqdm
import time

import re
from sklearn.preprocessing import SplineTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import RobustScaler
from eli5.sklearn import PermutationImportance
from sklearn.feature_selection import RFECV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import TimeSeriesSplit 
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import HuberRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.inspection import permutation_importance
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from xgboost import XGBRegressor
from sklearn.metrics import make_scorer
from sklearn.base import clone

from sklearn.exceptions import ConvergenceWarning
from sklearn.exceptions import DataConversionWarning
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=ConvergenceWarning)
simplefilter(action='ignore', category=DataConversionWarning)
simplefilter(action='ignore', category=UserWarning)

import xgboost as xgb
    
from collections import Counter

sns.set_theme(style="whitegrid")
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
def reinitialize():
    '''
    Used to initialize the datasets of predictors and dependent and rename the columns
    '''
    
    to_search = ['US', 'SENT', 'unc_bex', 'ra_bex', 'VIX', 'mcsi', r'slope\B', r'point0\b', 'convex']
    
    data = pd.read_excel("Data for ML.xlsx", header=0, parse_dates=['DateStru'], index_col=[1])
    
    Y = data.filter(like='logret')
    X = data.loc[:, data.columns.to_series().str.contains("|".join(to_search))]
    X = X.rename(columns={"USTBILLSECMARKET3MONTHDMIDDLERATE": "T-Bill", 
                  "unc_bex_AnnualVolPercentage": "unc_bex_%", 
                  "USInstitutional": "USInst",
                  "USIndividual": "USInd"})
    return X, Y

X, Y = reinitialize()

In [3]:

# dictionary with last observation available for test set
upp_bound = {x: Y[Y[x].isna()].index[0] - pd.to_timedelta(1, unit='w') for x in Y.columns[1:]} 
upp_bound["logret_1week"] = pd.to_datetime("2015-12-12")

# dictionary with first observation available for test set
interm_bound = {x: Y[Y[x].isna()].index[0] - pd.to_timedelta(53, unit='w') for x in Y.columns[1:]}

# dictionary with first observation available for evaluation set
#low_bound = {x: Y[Y[x].isna()].index[0] - pd.to_timedelta(53*2, unit='w') for x in Y.columns[1:]}


# important subsets for variables
pricing_kernel = ['point0', 'slopem2', 'slopep2', 'slopem5', 'slopep5', 'slopem10', 'slopep10', 'convexp2', 'convexm2']
except_kernel = [x for x in X.columns.to_list() if x not in pricing_kernel]  # only small gains if not including pricing kernel in standardization

In [4]:
def timer(name, start_time=None):
    '''
    Used to keep track of the time (in general, in particular for hypterparameter tuning)
    '''
    if not start_time:
        start_time=datetime.datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.datetime.now()-start_time).total_seconds(),3600)
        tmin, tsec = divmod(temp_sec,60)
        print("Execution time for", name, "is ", thour," h :", tmin,' m :', round(tsec,2), " s")

In [5]:
# Actually throughout the model implementations I used normal R2
# the reason is that the demeaned R2 is highly susceptible to standardization of the dependent
# and it did not seem to me a 'robust' metric to evaluate

# since I worked a lot on the standardizations, I used the normal R2 but for every function in evaluation
# there is a parameter 'score' that can be set to True and both the models and the metric collected
# will be Demeaned R2

def impl_r2(actual, predicted):
    '''
    Implemented R^2 computations corrected as in the paper (with the denominator not demeaned), 
    used to be comparable to the literature (even if we are at the market level and not in a panel of individual stocks)
    '''
    actual, predicted = np.array(actual), np.array(predicted)
    num = 0.0
    denom = 0.0 
    num = ((actual - predicted)**2).sum()
    denom = ((actual)**2).sum()  
    R2 = 1-(num/denom)
    return R2

impl_score = make_scorer(impl_r2, greater_is_better=True)

In [6]:
def yr_map(df, year, lq=True):
    '''
    Function to segment datasets on the basis of year (used below)
    '''
    if lq:
        result = df.loc[df.index.year <= year, :].copy(deep=True)
    else:
        result = df.loc[df.index.year == year, :].copy(deep=True)
    return result

def split_train_test(X, y, col, sc=False):   # rows of x passed is last year (before last observation in the dataset)
    '''
    Used to split train and test set (year by year) + take care of correct Standardization and avoid data leakages
    so that the scaler is refit only on the training data every time (and nothing of the test set is seen)
    
    If the splitting in done on the last year available for a dependent variable
    we take up to last observation available (and go back by one entire year), that is we take care of missing values
    '''
    year_max = X.index.year.max()
    if (col != "logret_1week") and (year_max == upp_bound[col].year):
        low_test, upp_test = interm_bound[col], upp_bound[col]
        X_test = X.loc[low_test:upp_test, :]  # Test set taken up to last available obs for each week lag
        y_test = y.loc[low_test:upp_test, col].to_frame()
        X_train = X.loc[:low_test, :]
        y_train = y.loc[:low_test, col].to_frame()
    else:
        X_test = yr_map(X, year_max, lq=False)
        y_test = y.loc[y.index.year == year_max, col].to_frame()
        X_train = yr_map(X, year_max-1)
        y_train = y.loc[y.index.year < year_max, col].to_frame()
    if sc:
        X_train, y_train, X_test, y_test = scale(sc, X_train, y_train, X_test, y_test)  # scaler function (below)
    return X_train, y_train, X_test, y_test


def scale(scaler, X_train, y_train, X_test=False, y_test=False):
    '''
    Used to scale appropriately on the basis of the scaler (dict) passed as parameter
    it allows independent scaling for a selected subset of variables and prevents data leakages (only train set is seen)
    Also only a set of data can be passed (to be completely standardized, used in PCA)
    
    scaler (dict): such as = {StandardScaler: [columns], MinMax: [columns]}
                   if columns=0 applied to Y, otherwise to X
    '''
    regress = X_train.columns
    for scal, variable in scaler.items():
        if variable == 0:  # dependent variable
            y_train = pd.DataFrame(data=scal.fit_transform(y_train), columns=y_train.columns)
            if type(y_test)==pd.DataFrame:
                y_test = pd.DataFrame(data=scal.transform(y_test), columns=y_test.columns)
        else:
            for ind in regress:
                if ind in variable:
                    X_train.loc[:, ind] = scal.fit_transform(X_train.loc[:, ind].to_numpy().reshape(-1, 1))
                    if type(X_test)==pd.DataFrame:
                        X_test.loc[:, ind] = scal.transform(X_test.loc[:, ind].to_numpy().reshape(-1, 1))
    if type(X_test)==pd.DataFrame:
        return X_train, y_train, X_test, y_test  # all are DataFrames
    return X_train, y_train

In [7]:
def log_transf():
    '''
    Implementation of log transformation for both Y and (all) X, !! dropping convex !! since extreme negative values (done on remaining columns)
    It was done following some RFE cases in which convex were not selected
    '''
    X_log, Y_log = reinitialize()
    X_log.drop(columns=["convexm2", "convexp2"], inplace=True)  # dropping convex
    X_min, Y_min = X_log.min().min(), Y_log.min().min()
    e = 10**(-7)  # epsilon
    scalar = Y_min if Y_min<X_min else X_min
    X_log = np.log(X_log.add(-scalar + e))
    Y_log = np.log(Y_log.add(-scalar + e))
    print("Scalar added is: ", scalar + e)  # still need to add small amount and shift so that both X and Y are positive
    return X_log, Y_log

def log_transf_nopk(dep=True):  # whether also Y or not
    '''
    Log transformation excluding PK variables (so that convex is retained) and dependent if specified
    dep: if True standardize also Y, otherwise not
    '''
    X, Y = reinitialize()
    X_pk = X.loc[:, pricing_kernel]
    X_macro = X.loc[:, except_kernel]
    
    e = 10**(-7)
    X_min = X_macro.min().min()
    Y_min = Y.min().min()
    
    if dep:
        scalar = Y_min if Y_min<X_min else X_min
        Y = np.log(Y.add(-scalar + e))
    else: 
        scalar = X_min
    X_macro = np.log(X_macro.add(-scalar + e))
    X = pd.concat([X_pk, X_macro], axis=1)
        
    print("Scalar added is: ", -scalar + e) 
    return X, Y


In [8]:
def transform(trans, X, predictors):
    '''
    Used for feature engineering on the regressors according to trans
    Entire X (as dataframe) is passed since columns can be independently standardized according to the dictionary
    trans (dict): similar to {PolynomialFeatures: [[new_columns], True], ....}  
                  if last term in list True we concat otherwise we overwrite X
    '''
    for transf, [list_new, truth] in list(trans.items()):
        X_transf = transf.fit_transform(X)
        if truth: # if additional True we concat
            X = pd.concat([X, pd.DataFrame(X_transf, columns=list_new, index=X.index)], axis=1)
            if type(predictors)!=list:
                predictors = predictors.to_list()
            predictors.extend(list_new)
        else:     # if false we overwrite
            X = pd.DataFrame(data=X_transf, columns=list_new, index=X.index)
            predictors = list_new
    return X, predictors

def rank_penalization(alpha):
    '''
    Used to print dependent with greater Penalization required (averaged over all alphas for each year)
    alpha (dict): {'logret_1week': tot, ....}
    '''
    alpha = dict(sorted(alpha.items(), key=lambda x: x[1], reverse=True))
    print(f"Ranking of dependent with greater amount of penalization required (agg.):")
    for i, v in enumerate(alpha): 
        print(f"{i}. {v}")


In [9]:
# FOR PLOTTING            

def graph_comparison(scores, threshold=-8, light=False):
    '''
    Used to plot all scores, for each variable and each year, divided in 3 groups for visibility 
    scores that are >0 or <threshold are plotted (with different graphics and making sure they are visible)
    
    scores (dict): {'logret_1week': [score_1, score_2, ....], 'logret_2week': ....}
    threshold (float/int): if scores are lower than this number they are visualized on the graph
    
    light (bool): using light=True if model is fit with light=True as well, which means that we are either plotting
                  list(range(2004, 2016)) that is all years of evaluation or list(range(2004, 2016, 2)) skipping some years
    
    '''
    
    years = [2004, 2006, 2008, 2010, 2012, 2014] if light else list(range(2004, 2016))
    positions = [0.5, 0.4, 0.3, 0.2, 0.1]
    to_plot = {x: [] for x in Y.columns}
    distance = positions.copy()
    tick_marks = np.arange(len(years))
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(28, 20))
    
    for key, value in scores.items():
        delay = int(re.search(r"\d+", key).group(0))
        if delay < 6:                        # 1, 2, 3, 4, 5
            ax = ax1
        elif delay == 6:
            ax = ax2
            distance = positions.copy()
        elif (delay > 6) and (delay < 13):    # 6, 7, 8, 9, 12
            ax = ax2
        elif delay == 24:
            ax = ax3
            distance = positions.copy()
        else:                              # 24, 36, 52, 78, 104
            ax = ax3
        prop = distance.pop(0)
        p = ax.plot(value, label=str(key).split("_")[1], linewidth=2)
        to_plot[key].extend([ax, prop, p[0].get_color()])
        for i, v in enumerate(value):
            if (v < threshold) or (v>0):   
                to_plot[key].append(i)
        ax.legend(loc='upper right', fontsize=14)
        ax.set_xticks(tick_marks)
        ax.set_xticklabels(years)
        ax.set_xlabel('Year of training set', fontsize=16)
        ax.set_ylabel(r'$R^2$', fontsize=18)
        ax.set_title('Performance of model', fontsize=18)
    
    for col, lst in to_plot.items():  # scores are plotted in order of labels at different propotion of the graph's height, to make them visible
        y_bound = lst[0].get_ylim()
        y = lst[1]*(y_bound[1]-y_bound[0]) + y_bound[0]
        for yr in lst[3:]:
            value = scores[col][int(yr)]
            w = "extra bold" if value>0 else "ultralight"
            f1 = "normal" if value>0 else "italic"
            f2 = "large" if value>0 else "medium"
            lst[0].text(yr, y, "{:.3f}%".format(value), c=lst[2],
                        weight=w, ha="center", fontstyle=f1, fontsize="large")  #.format(value[i])
    plt.show()
    

In [10]:
def sort_aggregate(aggregate=False, scores=False):
    '''
    Used to store in a dataframe and visualize 3 columns: - Agg Scores (avg), - Two last years (individual values)
    The dataframe is sorted according to - Agg Scores
    
    aggregate (dict): {'logret_1week': avg_scores, ...}
    scores (dict): {'logret_1week': [score_1, score_2, ...], ...}
    
    '''
    
    sor = aggregate if type(aggregate)==dict else scores
    d = sorted(sor, key=sor.get, reverse=True)
    store = {'Ranked models': [i for i in d]}
    if type(aggregate)==dict:
        store.update({'Agg Scores': [aggregate[i] for i in d]})
    if type(aggregate)==dict and type(scores)==dict:
        store.update({'': ['' for i in d]})
    if type(scores)==dict:
        try:
            store.update({'One to last': [scores[i][-2] for i in d]})
            store.update({'Last year': [scores[i][-1] for i in d]})
        except:
            store.update({'Last year': [scores[i] for i in d]})
    visual = pd.DataFrame(store)
    return visual  


def print_aggregate(aggregate):
    print("RANKING WITH AVERAGE")
    for k, i in enumerate(sorted(aggregate, key=aggregate.get, reverse=True)):
        print(f"{k}. Score for {i}: {aggregate[i]}")
        
        
def sort_coefficient(coeff):
    '''
    Used at the beginning to manually select stronger coefficients and run independent regressions
    Only last year coefficients are stored through train_recursive
    coeff (dict)
    
    '''
    predictors = list(X.columns)
    ranked_coeff = {y: 0 for y in Y.columns}
    for time_span, coef in coeff.items():
        idx = np.argsort(coef, axis=0)[::-1].flatten().tolist()
        ranked_predictors = [predictors[i] for i in idx]
        ranked_coeff[time_span] = ranked_predictors
    return pd.DataFrame(data = ranked_coeff)


def visualize_selected(selected, year=False, light=False, to_print=10):
    '''
    Used to visualize selected variables for each dependent variable
    selected (dict): {'logret_1week': [[select1_y1, select2_y1, ...], [select1_y2, select2_y2, ...], [],..], 'logret_2week': ...}
    
    year=int       (year from 2004 to 2015) -> we can visualize features selected for that year
    year=False      we visualize last year avaiable for each variable
    year=None       visualize for each dep most common selected variables among all years, 
                    sorted by appearence, and with absolute frequencies between ()
                    
    light (bool):    set to True if model is fit with light=True (to make sure invalid years cannot be asked for)
    
    to_print (int):  select how much of the ranking to print (for each dependent)
    
    '''
    prox = {}
    if year==False:
        prox = {k: v[-1] for k, v in selected.items()}
        prox = dict(sorted(prox.items(), key=lambda x: int(re.search(r"\d+", x[0]).group(0))))
    elif year==None:
        for dep, lst in selected.items():
            c = Counter()
            for dframe in lst:
                c.update(dframe[dep].tolist())
            common = [str(f) + " (" + str(s) + ")" for f, s in c.most_common(to_print)]   
            prox[dep] = pd.DataFrame({dep: common})
    else:
        years = [2004, 2006, 2008, 2010, 2012, 2014] if light else list(range(2004, 2016))
        try:
            idx = years.index(year)
        except:
            print('Not valid year')
        for k, v in selected.items():
            try:
                prox[k] = v[idx]
            except:
                continue
        #prox = {k: v[idx] for k, v in selected.items()}
        prox = dict(sorted(prox.items(), key=lambda x: int(re.search(r"\d+", x[0]).group(0))))
    c = pd.DataFrame()
    for variable, df in prox.items():
        c = pd.concat([c, df], axis=1)
    c.fillna(value="", inplace=True)
    return c

def plot_selected(selected, dependent):
    '''
    To plot how number of features selected change across the years (not used much)
    '''
    length = []
    for df in selected[dependent]:
        length.append(len(df[dependent].tolist()))
    tick_marks = np.arange(len(length))
    fig, ax = plt.subplots(1, 1, figsize=(15, 5))
    ax.plot(np.array(length), linewidth=2)
    ax.set_xticks(tick_marks)
    ax.set_xticklabels(list(range(2004, 2004+len(length))))
    ax.set_xlabel('Year of training set', fontsize=16)
    ax.set_ylabel('Number of features selected', fontsize=18)
    ax.set_title(f'Evolution of features selected for {dependent}', fontsize=18)  
    plt.show()
    
def compare_preds(Y, preds, col, y=2012, light=False):
    '''
    Used to plot the real Y values and the predicted values for a particular year and a particular dependent variable
    
    preds (dict)   as returned by all methods implemented (store all predictions for all years)
    col (str):     name of wanted columns (such as 'logret_1week')
    y (int):       wanted year
    light (bool):  to be set to true if model is fit with light
    
    '''
    lig = [2004, 2006, 2008, 2010, 2012, int(upp_bound[col].year)]
    if (light==True) and (y not in lig):
        return
    if y > upp_bound[col].year or y < 2004:
        return
    elif y == upp_bound[col].year:
        low_test, upp_test = interm_bound[col], upp_bound[col]
        real = Y.loc[low_test:upp_test, col]
    else:
        real = Y.loc[Y.index.year==y, col]
    if light:
        pred = preds[col][lig.index(y)]
    else:
        pred = preds[col][y-2004]
    fig = plt.figure(figsize=(15, 5))
    plt.title('Predictions of '+ str(col) + ' for year ' + str(y))
    plt.plot(real.index, real, linewidth=2, label='Real')
    plt.plot(real.index, pred, label='Predicted')
    plt.legend(fontsize="medium")
    plt.show()

In [11]:
   
def train_recursive(model, X, y, column=False, scal=False, score=False, ravel_=False, light=False, tr=False):   
    '''
    Used to train recursively a given model over all the years (up to each available year and observation) for each lagged week
    uses a specific model already runed
    
    model : sklearn estimator
    X, y : entire dataset as output by reinitialize
    scal (dict): calls the preceding functions to standardize the data, such as {StandardScaler: [columns], MinMax: [columns]}
    
    score (bool): !!!! if True demeaned R2 is used
    light (bool): !!! whether to compute for all years or skipping some
    tr (bool): to specifies if it is a tree or not
    columns (str): if computing for one specific column or for all recursively
    
    '''
    if not column:
        scores = {x: [] for x in y.columns}  
        aggregate = {x: 0 for x in y.columns}
        predic = {x: [] for x in y.columns}
        dependent = y.columns
        start_time=timer(None)
        if tr==False:
            coeff = {x: 0 for x in y.columns}
    else: 
        scores = {column: []}  
        aggregate = {column: 0}
        predic = {column: []}
        dependent = [column]
        if tr==False:
            coeff = {column: 0}
        y = pd.DataFrame(data=y, columns=[column], index=X.index)    
    span = range(2004, X.index.year.max()+1)
    if (light==True and column==True):
        span = [2004, 2006, 2008, 2010, 2012, int(upp_bound[column].year)]
    for year in span: # start with test set 2003-2004
        for time_span in dependent:
            if year > upp_bound[time_span].year:  # up to each available year
                continue
            X_train, y_train, X_test, y_test = split_train_test(yr_map(X, year), yr_map(y, year), col=time_span, sc=scal)
            trained = model.fit(X_train.values, y_train.values.ravel()) if ravel_==True else model.fit(X_train.values, y_train.values)
            preds = trained.predict(X_test.values)
            predic[time_span].append(preds) # storing predictions to plot
            if score:   # use demeaned score
                scores[time_span].append(impl_r2(y_test, preds))  # scores contains all scores in dimension n.years*lagged weeks
            else:
                scores[time_span].append(trained.score(X_test.values, y_test.values))
            if tr==False: # take coefficient only of the largest training set (max year=max available year) and not for trees
                if (year==2013) and (time_span=="logret_104week"):
                    coeff[time_span] = trained.coef_.reshape(-1, 1)  
                if (year==2014) and (time_span in ["logret_78week", "logret_52week", "logret_36week"]):
                    coeff[time_span] = trained.coef_.reshape(-1, 1)
                if (year==2015) and (time_span not in ["logret_104week", "logret_78week", "logret_52week", "logret_36week"]):
                    coeff[time_span] = trained.coef_.reshape(-1, 1)
    for key, value in scores.items():
        aggregate[key] = np.array(value).mean()
    if not column:
        timer('recursive training', start_time)
    if tr==False:
        return aggregate, scores, coeff, predic
    return aggregate, scores, predic

In [12]:

def feature_selection_full(estimat, X, Y, col=False, scaler=False, score=False, min_feat=1, light_=False, tree=False, ravel=False):
    '''
    Function to implement RFE recursively for each year for any given model, taking care of all scores, predictions, selected variables...
    using TimeSeriesSplit as a validation technique
    
    estimat is sklearn estimator
    scaler (dict) as above
    score (bool): if True using demeaned R2
    
    light_ (bool): if computing for all years or a subset of years
    '''
    if not col:
        aggregate = {x: 0 for x in Y.columns}   # mean across years for each dependent lagged week
        selected = {x: [] for x in Y.columns}  # to store variables selected
        scores = {x: [] for x in Y.columns}
        preds = {x: [] for x in Y.columns}
        dep = Y.columns
        start_time=timer(None)
    else:
        scores = {col: []}  
        aggregate = {col: 0}
        selected = {col: []}
        preds = {col: []}
        dep = [col]
        Y = pd.DataFrame(data=Y, columns=[col], index=X.index)
        
    span = list(range(2004, X.index.year.max()+1))
    if light_==True:
        span = list(range(2004, 2013, 2)) # light subset of available years
    predictors = X.columns
    year_min, year_max = X.index.year.min(), X.index.year.max()
    
    for dependent in tqdm(dep): 
        for year in span: # start with test set 2004
            if year > upp_bound[dependent].year:  # up to each available year
                continue
            if (light_==True) and (year==2004):
                span.append(int(upp_bound[dependent].year))
            X_train, y_train, X_test, y_test = split_train_test(yr_map(X, year), yr_map(Y, year), col=dependent, sc=scaler)
                
            generator = TimeSeriesSplit(n_splits = 3)  
            x = impl_score if score else 'r2'
            mod = clone(estimat) if tree not in [True, None] else PermutationImportance(estimat, scoring=x, n_iter=5, cv=TimeSeriesSplit(n_splits = 3))  # Permutation with cv (also) shows importance of features for generalization

            # Feature Selection 
            rfe = RFECV(estimator=mod, cv=generator, min_features_to_select=min_feat, scoring=x)  
            trained = rfe.fit(X_train.values, y_train.values.reshape(-1,)) if ravel==True else rfe.fit(X_train.values, y_train.values)

            # storing
            select_regress = np.array(predictors)[trained.get_support()].tolist()
            selected[dependent].append(pd.DataFrame({dependent: select_regress})) # storing
            
            pred = trained.predict(X_test.values)
            preds[dependent].append(pred) # storing predictions to plot
            
            sc = impl_r2(y_test, pred) if score else trained.score(X_test.values, y_test.values)
            scores[dependent].append(sc)  
            
        aggregate[dependent] = np.array(scores[dependent]).mean()
        if light_==True:
            span.pop(-1)   
            
    if not col:
        timer('feature selection', start_time)

    return selected, scores, aggregate, preds

#print(f"Out of {rfe.n_features_in_}, {rfe.n_features_} were selected for {dependent}")

In [13]:

def lasso(X, Y, col=False, scaler=False, light_=False, scor=False, thresh_prop=1.2):
    '''
    LassoCV implementation for feature engineering. Taking take that, if no variable is selected, SelectFromModel
    is used to raise progressively the threshold until few features are indeed selected
    
    light_ see above
    scor: seems that LassoCV uses R2 by default but if scor==True the score appended (and plotted) are the Demeaned R2
    
    thresh_prop: threshold used in a small loop inside to ensure that some variables are selected (the higher, the quicker)
    '''
    if not col:
        aggregate = {x: 0 for x in Y.columns}   # mean across years for each dependent lagged week
        selected = {x: [] for x in Y.columns}  # to store variables selected
        scores = {x: [] for x in Y.columns}
        preds = {x: [] for x in Y.columns}
        alpha = {x: [] for x in Y.columns}
        dep = Y.columns
        start_time=timer(None)
    else:
        scores = {col: []}  
        aggregate = {col: 0}
        selected = {col: []}
        preds = {col: []}
        alpha = {col: []}
        dep = [col]
        Y = pd.DataFrame(data=Y, columns=[col], index=X.index)
    
    span = list(range(2004, X.index.year.max()+1))
    if light_==True:
        span = list(range(2004, 2013, 2))
        
    predictors = X.columns
    year_min, year_max = X.index.year.min(), X.index.year.max()
  
    for dependent in tqdm(dep):
        for year in span: # start with test set 2003-2004
            if year > upp_bound[dependent].year:  # up to each available year
                continue
            if (light_==True) and (year==2004):
                span.append(int(upp_bound[dependent].year))
            X_train, y_train, X_test, y_test = split_train_test(yr_map(X, year), yr_map(Y, year), col=dependent, sc=scaler)
            
            generator = TimeSeriesSplit(n_splits = 5)
            # fitting
            lasso = LassoCV(cv=generator, selection='random', max_iter=100000, eps=1e-3, tol=5e-4)   # tol=0.0005
            trained = lasso.fit(X_train, y_train)  #trained = lasso.fit(X_train, y_train.values.ravel())
            
            # storing
            alpha_ = trained.alpha_
            alpha[dependent].append(alpha_)
            select_regress = np.array(predictors)[np.nonzero(trained.coef_)[0]].tolist()

            if len(select_regress) < 1:   # to make sure strongest variables selected even if initially discarded
                thresh = "median"
                #print("IN LOOP")
                while len(select_regress) not in [1, 2, 3]:
                    sfm = SelectFromModel(LinearRegression(), threshold=thresh).fit(X_train, y_train)
                    select_regress = np.array(predictors)[sfm.get_support()].tolist()
                    #print(f"selected: {len(select_regress)}")
                    thresh = sfm.threshold_ * thresh_prop
                    if len(select_regress)==0:
                        #print("STUCK")
                        thresh = "mean"
                #print("EXIT LOOP")
                X_test = sfm.transform(X_test)
                trained = sfm.estimator_.fit(sfm.transform(X_train), y_train)

            selected[dependent].append(pd.DataFrame({dependent: select_regress})) # storing
        
            pred = trained.predict(X_test)  #.values
            preds[dependent].append(pred) # storing predictions to plot
            
            sc = impl_r2(y_test, pred) if scor else trained.score(X_test, y_test) # trained.score(X_test, y_test.values)
            scores[dependent].append(sc)  
        
        aggregate[dependent] = np.array(scores[dependent]).mean()
        if light_==True:
            span.pop(-1)
            
    alpha = {k: np.array(v).mean() for k, v in alpha.items()}
    rank_penalization(alpha)
    timer('lasso', start_time)
    
    return selected, scores, aggregate, preds


In [14]:

def grid_search(X, Y, model, params_light, params_hard, scal_=False, sco=False, selection=False, _light_=False, tree_=False):
    '''
    Used for hyper-parameter tuning given a list
    
    (only if selection == True)
                1) Iniziatially RandomizedSearchCV is used on data up to year 2004 (the first training set)
                to (lightly) select appropriate hyperparameters and avoid conditioning feature selection
                2) Then Feature selection is performed upon this (lightly) optimized model
    
    (both if selection==True or False)
                Eventually GridSearchCV is fitted to squeeze the model predictability, for each year and dependent variable
    
    params_light (dict): list of hyperparameters to be (lightly) selected using 2001-2003 as test set
    params_hard (dict): full list of hyperparameters to be evaluated once features are selected
    
    scal_ (dict): dictionary with scalers (see above)
    selection (bool): determines whether feature selection is performed or not
    
    _light_ (bool): whether to use all available years or the subset (2004, 2006, 2008...)
    
    tree_: if None a single tree is used (say Decision Tree) so node_count and depth of fitted tree is stored
            if True/False is an ensemble Algo and these parameters are not stored (as we have multiple trees)
    '''
    
    agg = {x: 0 for x in Y.columns} 
    selected = {x: [] for x in Y.columns}
    scores = {x: [] for x in Y.columns}
    preds = {x: [] for x in Y.columns}
    dep = Y.columns
    params_grid = {x: [] for x in Y.columns}
    params_tree = {x: [] for x in Y.columns}
    
    predictors = X.columns
    year_min, year_max = X.index.year.min(), X.index.year.max()  
    x = impl_score if sco else 'r2'
    start_time=timer(None)  # timer start
    
    span = list(range(2004, X.index.year.max()+1))
    if _light_==True:
        span = list(range(2004, 2013, 2))
    
    for dependent in tqdm(Y.columns):        
        for year in span: # start with test set 2003-2004
            if year > upp_bound[dependent].year:  # up to each available year
                continue
            if (_light_==True) and (year==2004):
                span.append(int(upp_bound[dependent].year))
            if selection:
                if year==2004:
                    X_tr, Y_tr, X_te, Y_te = split_train_test(yr_map(X, year), yr_map(Y, year), col=dependent, sc=scal_)
                    _gen = TimeSeriesSplit(n_splits = 3) 
                    first = RandomizedSearchCV(estimator=model, cv=_gen, param_distributions=params_light, n_iter=150, scoring=x, n_jobs=-1, verbose=0)
                    trained = first.fit(X_tr, Y_tr.values.ravel())
                    first_mod = trained.best_estimator_ # clone(model).set_params(**lax.best_params_)
                if year != 2004:
                    X_tr, Y_tr, X_te, Y_te = split_train_test(yr_map(X, year), yr_map(Y, year), col=dependent, sc=scal_)
                    
                gen_ = TimeSeriesSplit(n_splits = 5)  
                mod = first_mod if tree_ not in [True, None] else PermutationImportance(first_mod, scoring=x, n_iter=3, cv=TimeSeriesSplit(n_splits = 5))  # Permutation with cv (also) shows importance of features for generalization
                
                # Feature Selection 
                rfe = RFECV(estimator = mod, cv=gen_, min_features_to_select=1, scoring=x)  
                try:
                    trained = rfe.fit(X_tr, Y_tr.values)      # if ravel==True else rfe.fit(X_tr.values, Y_tr.values) .values.reshape(-1,)
                    
                    # storing
                    select_regress = np.array(predictors)[trained.get_support()].tolist()
                    selected[dependent].append(pd.DataFrame({dependent: select_regress})) # storing        

                    X_tr = pd.DataFrame(data=rfe.transform(X_tr), columns=select_regress, index=X_tr.index)
                    X_te = pd.DataFrame(data=rfe.transform(X_te), columns=select_regress, index=X_te.index)
                except ValueError:
                    print(f"RFE for {dependent} year {year} did not converge")
                    selected[dependent].append(pd.DataFrame({dependent: np.array(predictors).tolist()}))
                    X_tr, Y_tr, X_te, Y_te = split_train_test(yr_map(X, year), yr_map(Y, year), col=dependent, sc=scal_)
            else:
                X_tr, Y_tr, X_te, Y_te = split_train_test(yr_map(X, year), yr_map(Y, year), col=dependent, sc=scal_)
            
            generator = TimeSeriesSplit(n_splits = (year - 2004 + 2)) # year 2004 starts with n_splits=2 then +1 every yea
            grid = GridSearchCV(estimator=model, cv=generator, param_grid=params_hard, scoring=x, n_jobs=-1, verbose=0, refit=True)
            trained = grid.fit(X_tr, Y_tr.values.ravel())
            params_grid[dependent].append(grid.best_params_)
            
            pred = trained.predict(X_te)
            preds[dependent].append(pred) # storing predictions to plot
            
            sc = impl_r2(y_test, pred) if sco else grid.score(X_te, Y_te.values)
            scores[dependent].append(sc) 
            
            if tree_==None:
                tr = trained.best_estimator_.tree_
                params_tree[dependent].append([tr.node_count, tr.max_depth])            
            
        agg[dependent] = np.array(scores[dependent]).mean()
        if _light_==True:
            span.pop(-1)     
            
    timer('grid search', start_time) # timer
    print("")
    
    if selection: 
        if tree_==None:
            return selected, agg, scores, [params_grid, params_tree], preds   # preds only if selection
        return selected, agg, scores, params_grid, preds
    if tree_==None:
        return agg, scores, [params_grid, params_tree], preds
    return agg, scores, params_grid, preds
    
    

In [15]:
def print_param(params, params_tree=False):
    '''
    Function used to plot the set of parameters chosen by the (hard) GridSearch
    
    if params_tree is a dict: print also most common n.nodes and tree_depth for the fitted tree (for each dependent)
    '''
    for i, (dep, param) in enumerate(params.items()):
        final = []
        store = [[] for _ in range(len(param[0]))]
        
        if params_tree:
                node_count = [params_tree[dep][x][0] for x in range(len(params_tree[dep]))]
                depth_count = [params_tree[dep][x][1] for x in range(len(params_tree[dep]))]
                node_mean = np.array(node_count).mean(dtype=int)
                depth_mean = np.array(depth_count).mean(dtype=int)
                
        for p_year in param:
            for i, (p, value) in enumerate(p_year.items()):
                store[i].append(p + ": " + str(value))
                
        for var in store:
            c = Counter(var)
            final.append(c.most_common(1)[0][0])
        
        a = f"\t \t \t \t Fitted model's avg. node counts: {node_mean} \t \t avg. depth count = {depth_mean}" if params_tree else ""
        print(f"\nDependent {dep}:" + a)
        for i in final:
            print(f"\t \t Most common {i}")
        

### Spline transformation

In [16]:

def f_spline(columns, X, knots=3, degree=2):  # columns = ['var', 'ra']
    '''
    Function to find generated splines from a set of columns (to standardize them)
    '''
    f = []
    for group in columns:
        for col in group:
            index = X.columns.to_list().index(col)
            low = index*(knots+degree-1)+1
            names = [str(i) for i in range(low, low + (knots+degree-1))]
            f += names
    return f

def s_spline(group, X, knots=3, degree=2, vis=False):            # group = [1, 2, 3, 4]
    '''
    Inverse function of above
    '''
    for spline in group:
        index = int((int(spline)-1)/(knots+degree-1))   # inverse of spline creation to find original linear regressor
        col = X.columns.to_list()[index]
        if vis:
            print(f"{spline} is a spline transformation of {col}")
    return
    
def print_common(store, X, to_print=5, kn=3, deg=2):  # store is database of selected regressors, X is common X not standardized
    '''
    Function used to find the most common splines in the Dataset of selected variables ( dependent on to_print (int) )
    and print their origin (i.e. which dependent variable they are the Spline transformation of)
    '''
    e = Counter()
    for dep, lst in store.items():
        c = Counter()
        d = Counter()
        for dframe in lst:
            c.update(dframe[dep].tolist())
        for var, freq in c.items():
            try:
                x = int(var)
                d.update({var: freq})
            except:
                continue
        c = {f:s for (f, s) in d.most_common(15)}
        e.update(c)
    fin = [a for (a, l) in e.most_common(to_print)]
    s_spline(fin, X, kn, deg, vis=True)
    return


### Polynomial features

In [17]:
def f_int(to_find, col_names):  # to_find = ['var', 'ra'], col_names=above
    '''
    Used to find all interaction generated from a given set of columns
    '''
    f = set()
    for group in to_find:
        for col in group:
            names = [inter for inter in col_names if re.search(col, inter)]
            f.update(names)
    return list(f)

### PCA

In [18]:
def explain_component(comp, columns):
    '''
    Used to explain the composition (in terms of % of direction) of the Principal Component generated by the PCA transformation
    derived from the definition of Principal Component and the way they are derived
    '''
    a = [(np.round_((comp[i]**2)*100, decimals=1), columns[i]) for i in range(len(comp))]
    a.sort(key = lambda x: x[0], reverse=True)
    
    n, i = 0, 0
    while n<70:
        print(f"{a[i][0]} % of component's direction: {a[i][1]}")
        n += a[i][0]
        i += 1
        
        
def print_pca(store, pca_var, cols, to_print=5):  # store is database of selected regressors, X is common X not standardized
    '''
    Similar to above but for PCA, print most common PComponents selected out of the store with all
    the selections and their composition in terms of % of original regressors
    '''
    # print pca from store (descending order for frequency)
    e = Counter()
    for dep, lst in store.items():
        c = Counter()
        d = Counter()
        for dframe in lst:
            c.update(dframe[dep].tolist())
        for var, freq in c.items():
            if re.search(r'\bPC', var): 
                d.update({var: freq})
        c = {f:s for (f, s) in d.most_common(15)}
        e.update(c)
    fin = [int(re.search(r'\d+' ,a).group(0)) for (a, l) in e.most_common(to_print)]
    for i in fin:
        print("PC" + str(i))
        explain_component(pca_var.components_[i-1], columns=cols)
        print("")
    return


def fit_pca(X, col=False, perc=0.9): 
    '''
    Used to fit PCA to a subset of columns (and Standardizing them first to mean 0 and variance 1)
    keeping the remaining set of regressors (not inside 'col') linear
    
    col (list): list of columns' names to be transformed
    perc (float):  % of total variance of selected columns to explain
    '''
    pca = PCA(n_components=perc, svd_solver='full')
    if col:
        oth = [X.columns.to_list()[i] for i in range(X.shape[1]) if X.columns.to_list()[i] not in col]
        X_sel = pd.DataFrame(data=StandardScaler().fit_transform(X.loc[:, col]), columns=col)
        X_oth = X.loc[:, oth]
        pca.fit(X_sel)
        pc_names = ['PC' + str(i+1) for i in range(pca.n_components_)]
        X_pca = pd.DataFrame(data = pca.transform(X_sel), columns=pc_names, index=X.index)
        X = pd.concat([X_pca, X_oth], axis=1)
    else:
        col = X.columns.to_list()
        X = pd.DataFrame(data=StandardScaler().fit_transform(X), columns=col, index=X.index)
        pca.fit(X)
        pc_names = ['PC' + str(i+1) for i in range(pca.n_components_)]
        X = pd.DataFrame(data = pca.transform(X), columns=pc_names, index=X.index)
    print(f"{pca.n_components_} components explain {perc: .0%} of the variance for the selected columns {col} \n")
    for i in range(pca.n_components_):
        print(f"The {i+1}° explains {pca.explained_variance_ratio_[i]: .1%}%")
    return X, pca
        

#### Additional Code saved

X, Y = reinitialize()
X_tr, Y_tr, X_te, Y_te = split_train_test(yr_map(X, 2006), yr_map(Y, 2006), col='logret_1week')
X_tr.index

print(X_tr.index[0])
print(X_tr.index[-1])

generator = TimeSeriesSplit(n_splits = 3)
print(X_tr.shape)
idx = X_tr.index
print('Index PRE', idx[0], idx[-1])
for train_index, test_index in generator.split(X_tr):
    print("TRAIN:", train_index, "\nTEST:", test_index)
    X_train, X_test = X_tr.iloc[train_index, :], X_tr.iloc[test_index, :]
    print(X_train.shape)
    print(X_test.shape)
    idx = X_test.index
    print('Index POST', idx[0], idx[-1])
    print("")


In [19]:
def feature_selection_light(estimat, X, Y, col=False, scaler=False, score=False, min_feat=1):  #score True if using demeaned
    if not col:
        aggregate = {x: 0 for x in Y.columns}   # mean across years for each dependent lagged week
        selected = {x: 0 for x in Y.columns}  # to store variables selected
        scores = {x: [] for x in Y.columns}
        dep = Y.columns
        start_time=timer(None)
    else:
        scores = {col: []}  
        aggregate = {col: 0}
        selected = {x: 0 for x in Y.columns}
        dep = [col]
        Y = pd.DataFrame(data=Y, columns=[col], index=X.index)
    predictors = X.columns
    year_min, year_max = X.index.year.min(), X.index.year.max()
    for dependent in dep: 
        y_max = upp_bound[dependent].year
        X_tr, Y_tr, X_te, Y_te = split_train_test(yr_map(X, y_max), yr_map(Y, y_max), col=dependent, sc=scaler)
        generator = TimeSeriesSplit(n_splits = (y_max-year_min))
        # scoring for selection
        if score:  # using de-meaned
            rfe = RFECV(estimator=estimat, cv=generator, min_features_to_select=min_feat, scoring=impl_score)
        else:
            rfe = RFECV(estimator=estimat, cv=generator, min_features_to_select=min_feat, scoring='r2')
        # training and filtering
        trained = rfe.fit(X_tr.values, Y_tr)   # Feature Selection done on largest training set
        cv_results = pd.DataFrame(rfe.cv_results_)
        select_regress = np.array(predictors)[trained.get_support()].tolist()
        number = int(len(select_regress)) - min_feat
        scores[dependent]= cv_results.iloc[number, 2:].to_list()
        scores[dependent].append(rfe.score(X_te.values, Y_te))
        aggregate[dependent] = np.array(scores[dependent]).mean()
        # storing
        selected[dependent] = pd.DataFrame({dependent: select_regress}) # storing
    if not col:
        timer('feature selection (light)', start_time)
    store = visualize_selected(selected)
    return store, scores, aggregate

In [20]:
# LASSO WITH SCORES ONLY FOR LAST YEAR (LARGEST TRAINING SET)

def lasso_light(X, Y, scaler=False, trans=False):  
    alpha = []
    aggregate = {x: [] for x in Y.columns}
    selected = {x: 0 for x in Y.columns}  # to store variables selected
    predictors, dep = X.columns, Y.columns
    year_min, year_max = X.index.year.min(), X.index.year.max()
    if trans:
        X, predictors = transform(trans, X, predictors)  # feature engineering
    start_time=timer(None)
    for dependent in Y.columns: 
        y_max = upp_bound[dependent].year
        X_tr, Y_tr, X_te, Y_te = split_train_test(yr_map(X, y_max), yr_map(Y, y_max), col=dependent, sc=scaler)
        generator = TimeSeriesSplit(n_splits = (y_max - year_min))
        # fitting
        lasso = LassoCV(cv=generator, selection='cyclic', max_iter=250000, eps=1e-5, tol=5e-4)   # tol=0.0001, eps=1e-3
        lasso.fit(X_tr, Y_tr.values.ravel())
        alpha.append(lasso.alpha_)
        # storing
        sel_reg = np.array(predictors)[np.nonzero(lasso.coef_)[0]].tolist()
        aggregate[dependent] = lasso.score(X_te, Y_te.values.ravel())
        # storing
        selected[dependent] = pd.DataFrame({dependent: sel_reg}) # storing
        
    timer('lasso (light)', start_time)
    rank_penalization(alpha)
    
    store = pd.DataFrame()
    for variable, df in selected.items():
        store = pd.concat([store, df], axis=1)
    store.fillna(value="", inplace=True)   
    
    return store, aggregate
